In [35]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

In [37]:
def cargar_imagen(direccion_imagen, tamanio=(600, 800)):
    img = Image.open(direccion_imagen).convert('L')
    img = img.resize(tamanio)
    img_array = np.array(img) / 255.0
    return img_array.flatten()

In [39]:
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivada(x):
    return x * (1 - x)

In [41]:
# Red neuronal
class Red_Neuronal:
    def __init__(self, tamanio_entrada, tamanios_ocultos, tamanio_salida):

        self.tamanio_entrada = tamanio_entrada
        self.tamanios_ocultos = tamanios_ocultos
        self.tamanio_salida = tamanio_salida
        
        self.pesos = []
        self.sesgos = []
        
        tamanio_anterior = tamanio_entrada
        for tamanio_oculto in tamanios_ocultos:
            self.pesos.append(np.random.randn(tamanio_anterior, tamanio_oculto).astype(np.float32))
            self.sesgos.append(np.zeros((1, tamanio_oculto), dtype=np.float32))
            tamanio_anterior = tamanio_oculto
        
        self.pesos.append(np.random.randn(tamanio_anterior, tamanio_salida).astype(np.float32))
        self.sesgos.append(np.zeros((1, tamanio_salida), dtype=np.float32))

    def propagacion_adelante(self, X):
        self.a = [X]
        self.z = []

        for i in range(len(self.tamanios_ocultos)):
            z = np.dot(self.a[-1], self.pesos[i]) + self.sesgos[i]
            a = sigmoid(z)
            self.z.append(z)
            self.a.append(a)
        
        z_salida = np.dot(self.a[-1], self.pesos[-1]) + self.sesgos[-1]
        salida = sigmoid(z_salida)
        self.z.append(z_salida)
        self.a.append(salida)
        
        return salida

    def propagacion_atras(self, X, y, tasa_aprendizaje=0.01):
        error_salida = y - self.a[-1]  # Error en la capa de salida
        salida_delta = error_salida * sigmoid_derivada(self.a[-1])
        
        self.pesos[-1] += self.a[-2].T.dot(salida_delta) * tasa_aprendizaje
        self.sesgos[-1] += np.sum(salida_delta, axis=0, keepdims=True) * tasa_aprendizaje
        
        delta = salida_delta
        for i in range(len(self.tamanios_ocultos) - 1, -1, -1):
            delta = delta.dot(self.pesos[i + 1].T) * sigmoid_derivada(self.a[i + 1])
            self.pesos[i] += self.a[i].T.dot(delta) * tasa_aprendizaje
            self.sesgos[i] += np.sum(delta, axis=0, keepdims=True) * tasa_aprendizaje

    def entrenamiento(self, X, y, epochs, tasa_aprendizaje):
        for epoch in range(epochs):
            self.propagacion_adelante(X)
            self.propagacion_atras(X, y, tasa_aprendizaje)
            if epoch % 200 == 0:
                loss = np.mean(np.square(y - self.a[-1]))  # Error cuadrático medio
                print(f"Epoch {epoch}, Loss: {round(loss, 5)}")

    def prediccion(self, X):
        return self.propagacion_adelante(X)

In [43]:
direccion_imagen_entrenamiento = 'C:/Users/joel_/OneDrive/Documents/JOEL/UNAH/2025/I PAC/IA/Proyecto/img/Entrenamiento.jpeg'
#direccion_imagen_prueba = 'C:/Users/joel_/OneDrive/Documents/JOEL/CV/1.jpeg'  # Ruta de la imagen de prueba
direccion_imagen_prueba = 'C:/Users/joel_/OneDrive/Documents/JOEL/UNAH/2025/I PAC/IA/Proyecto/img/Prueba_1.jpg'  # Ruta de la imagen de prueba

X_train = cargar_imagen(direccion_imagen_entrenamiento)
X_test = cargar_imagen(direccion_imagen_prueba)

# Etiqueta para la imagen de entrenamiento (1 si la imagen es la que esperamos, 0 si no)
y_train = np.array([[1]])

In [45]:
# Creacion de la red neuronal
tamanio_entrada = 600 * 800  # 480,000 píxeles
tamanios_ocultos = [300, 200, 100, 50, 25]  # Número arbitrario de neuronas en la capa oculta
tamanio_salida = 1  # Capa de salida

In [47]:
nn = Red_Neuronal(tamanio_entrada, tamanios_ocultos, tamanio_salida)

In [49]:
# Entrenar la red neuronal
nn.entrenamiento(X_train.reshape(1, -1), y_train, epochs=1000, tasa_aprendizaje=0.01)

Epoch 0, Loss: 0.78406
Epoch 200, Loss: 0.00961
Epoch 400, Loss: 0.00515
Epoch 600, Loss: 0.00355
Epoch 800, Loss: 0.00273


In [51]:
# Realizar predicciones
prediction_train = nn.prediccion(X_train.reshape(1, -1))
prediction_test = nn.prediccion(X_test.reshape(1, -1))

In [52]:
print(f"Predicción en la imagen de entrenamiento: {prediction_train}")
print(f"Predicción en la imagen de prueba: {prediction_test}")

# Mostrar resultados
if prediction_test < 0.5:
    print("No se reconoce la imagen de prueba.")
else:
    print("Se reconoce la imagen de prueba.")

Predicción en la imagen de entrenamiento: [[0.95291196]]
Predicción en la imagen de prueba: [[0.52096108]]
Se reconoce la imagen de prueba.
